In [1]:
import os
import numpy as np
import pandas as pd
import data_helpers
import pickle
from data_helpers import TrainValTensorBoard
from keras.callbacks import EarlyStopping
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.layers import Input, Embedding, Activation, Flatten, Dense, Concatenate
from keras.layers import Conv1D, MaxPooling1D, Dropout
from keras.models import Model


Using TensorFlow backend.


In [2]:
#==================Preprocess===================

# Load data
csv = '../data/twitter/clean_tweet.csv'
df = pd.read_csv(csv, index_col=0)
print(df.head())

# Delete Null row
df = df.dropna()
print(df.target.value_counts())

df['target'] = df['target'].map({0: 0, 4: 1})

x_text = df['text'].values
y = df['target'].values
y = to_categorical(y)


/anaconda3/envs/py36/lib/python3.6/site-packages/numpy/lib/arraysetops.py:466: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


                                                text  target
0  awww that s a bummer you shoulda got david car...       0
1  is upset that he can t update his facebook by ...       0
2  i dived many times for the ball managed to sav...       0
3     my whole body feels itchy and like its on fire       0
4  no it s not behaving at all i m mad why am i h...       0
0    798503
4    798250
Name: target, dtype: int64


In [3]:
# Tokenizer
tk = Tokenizer(num_words=None, char_level=True, oov_token='UNK')
tk.fit_on_texts(x_text)

In [4]:
tk.word_index

{' ': 1,
 'UNK': 28,
 'a': 5,
 'b': 21,
 'c': 18,
 'd': 12,
 'e': 2,
 'f': 19,
 'g': 16,
 'h': 9,
 'i': 6,
 'j': 24,
 'k': 22,
 'l': 11,
 'm': 13,
 'n': 7,
 'o': 3,
 'p': 20,
 'q': 27,
 'r': 10,
 's': 8,
 't': 4,
 'u': 14,
 'v': 23,
 'w': 17,
 'x': 25,
 'y': 15,
 'z': 26}

In [5]:
# Convert string to index
sequences = tk.texts_to_sequences(x_text)

In [8]:
print(len(x_text[0]))
print(x_text[0])
print(len(sequences[0]))
print(sequences[0])

71
awww that s a bummer you shoulda got david carr of third day to do it d
71
[5, 17, 17, 17, 1, 4, 9, 5, 4, 1, 8, 1, 5, 1, 21, 14, 13, 13, 2, 10, 1, 15, 3, 14, 1, 8, 9, 3, 14, 11, 12, 5, 1, 16, 3, 4, 1, 12, 5, 23, 6, 12, 1, 18, 5, 10, 10, 1, 3, 19, 1, 4, 9, 6, 10, 12, 1, 12, 5, 15, 1, 4, 3, 1, 12, 3, 1, 6, 4, 1, 12]


In [9]:
import pandas as pd  
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [10]:
cols = ['sentiment','id','date','query_string','user','text']

In [11]:
df = pd.read_csv("../data/twitter/training.1600000.processed.noemoticon.csv", header=None, names=cols, encoding='latin1')

In [12]:
df.drop(['id','date','query_string','user'], axis=1, inplace=True)

In [13]:
df['sentiment'] = df['sentiment'].map({0: 0, 4: 1})

In [14]:
df.head()

,sentiment,text
0,0,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,is upset that he can't update his Facebook by ...
2,0,@Kenichan I dived many times for the ball. Man...
3,0,my whole body feels itchy and like its on fire
4,0,"@nationwideclass no, it's not behaving at all...."


In [15]:
df.tail()

,sentiment,text
1599995,1,Just woke up. Having no school is the best fee...
1599996,1,TheWDB.com - Very cool to hear old Walt interv...
1599997,1,Are you ready for your MoJo Makeover? Ask me f...
1599998,1,Happy 38th Birthday to my boo of alll time!!! ...
1599999,1,happy #charitytuesday @theNSPCC @SparksCharity...


In [19]:
from bs4 import BeautifulSoup 
import re 

# Data Preparation 1: HTML decoding
pat1 = r'@[A-Za-z0-9]+'   # @mention
pat2 = r'https?://[A-Za-z0-9./]+' # URL
combined_pat = r'|'.join((pat1, pat2))

def tweet_cleaner(text):
    soup = BeautifulSoup(text, 'lxml')  # HTML decoding
    souped = soup.get_text()
    stripped = re.sub(combined_pat, '', souped) # URL, @mention
    try:
        clean = stripped.replace(u'ï¿½', ' ') # Latin character encoding
    except:
        clean = stripped
#     letters_only = re.sub("[^a-zA-Z]", " ", clean)  # We reserve punctuation
    lower_case = clean.lower()
    # During the letters_only process two lines above, it has created unnecessay white spaces,
    # I will tokenize and join together to remove unneccessary white spaces
#     words = tok.tokenize(lower_case)
    return lower_case

In [20]:
testing = df.text[:100]
testing

0     @switchfoot http://twitpic.com/2y1zl - Awww, t...
1     is upset that he can't update his Facebook by ...
2     @Kenichan I dived many times for the ball. Man...
3       my whole body feels itchy and like its on fire 
4     @nationwideclass no, it's not behaving at all....
5                         @Kwesidei not the whole crew 
6                                           Need a hug 
7     @LOLTrish hey  long time no see! Yes.. Rains a...
8                  @Tatiana_K nope they didn't have it 
9                             @twittera que me muera ? 
10          spring break in plain city... it's snowing 
11                           I just re-pierced my ears 
12    @caregiving I couldn't bear to watch it.  And ...
13    @octolinz16 It it counts, idk why I did either...
14    @smarrison i would've been the first, but i di...
15    @iamjazzyfizzle I wish I got to watch it with ...
16    Hollis' death scene will hurt me severely to w...
17                                 about to file

In [21]:
test_result = []
for t in testing:
    test_result.append(tweet_cleaner(t))

In [22]:
test_result

["  - awww, that's a bummer.  you shoulda got david carr of third day to do it. ;d",
 "is upset that he can't update his facebook by texting it... and might cry as a result  school today also. blah!",
 ' i dived many times for the ball. managed to save 50%  the rest go out of bounds',
 'my whole body feels itchy and like its on fire ',
 " no, it's not behaving at all. i'm mad. why am i here? because i can't see you all over there. ",
 ' not the whole crew ',
 'need a hug ',
 " hey  long time no see! yes.. rains a bit ,only a bit  lol , i'm fine thanks , how's you ?",
 "_k nope they didn't have it ",
 ' que me muera ? ',
 "spring break in plain city... it's snowing ",
 'i just re-pierced my ears ',
 " i couldn't bear to watch it.  and i thought the ua loss was embarrassing . . . . .",
 ' it it counts, idk why i did either. you never talk to me anymore ',
 " i would've been the first, but i didn't have a gun.    not really though, zac snyder's just a doucheclown.",
 ' i wish i got to wat

In [23]:
nums = [0,400000,800000,1200000,1600000]

In [24]:
%%time
print("Cleaning and parsing the tweets...\n")
clean_tweet_texts = []
for i in range(nums[0],nums[1]):
    if( (i+1)%10000 == 0 ):
        print("Tweets %d of %d has been processed" % ( i+1, nums[1] ))
    clean_tweet_texts.append(tweet_cleaner(df['text'][i]))

Cleaning and parsing the tweets...

Tweets 10000 of 400000 has been processed
Tweets 20000 of 400000 has been processed
Tweets 30000 of 400000 has been processed
Tweets 40000 of 400000 has been processed
Tweets 50000 of 400000 has been processed
Tweets 60000 of 400000 has been processed
Tweets 70000 of 400000 has been processed
Tweets 80000 of 400000 has been processed
Tweets 90000 of 400000 has been processed
Tweets 100000 of 400000 has been processed
Tweets 110000 of 400000 has been processed
Tweets 120000 of 400000 has been processed
Tweets 130000 of 400000 has been processed
Tweets 140000 of 400000 has been processed
Tweets 150000 of 400000 has been processed
Tweets 160000 of 400000 has been processed
Tweets 170000 of 400000 has been processed
Tweets 180000 of 400000 has been processed
Tweets 190000 of 400000 has been processed
Tweets 200000 of 400000 has been processed
Tweets 210000 of 400000 has been processed
Tweets 220000 of 400000 has been processed
Tweets 230000 of 400000 has

In [25]:
%%time
print("Cleaning and parsing the tweets...\n")
for i in range(nums[1],nums[2]):
    if( (i+1)%10000 == 0 ):
        print("Tweets %d of %d has been processed" % ( i+1, nums[2] ))
    clean_tweet_texts.append(tweet_cleaner(df['text'][i]))

Cleaning and parsing the tweets...

Tweets 410000 of 800000 has been processed
Tweets 420000 of 800000 has been processed
Tweets 430000 of 800000 has been processed
Tweets 440000 of 800000 has been processed
Tweets 450000 of 800000 has been processed
Tweets 460000 of 800000 has been processed
Tweets 470000 of 800000 has been processed
Tweets 480000 of 800000 has been processed
Tweets 490000 of 800000 has been processed
Tweets 500000 of 800000 has been processed
Tweets 510000 of 800000 has been processed
Tweets 520000 of 800000 has been processed
Tweets 530000 of 800000 has been processed
Tweets 540000 of 800000 has been processed
Tweets 550000 of 800000 has been processed
Tweets 560000 of 800000 has been processed
Tweets 570000 of 800000 has been processed
Tweets 580000 of 800000 has been processed
Tweets 590000 of 800000 has been processed
Tweets 600000 of 800000 has been processed
Tweets 610000 of 800000 has been processed
Tweets 620000 of 800000 has been processed
Tweets 630000 of 8

In [26]:
%%time
print("Cleaning and parsing the tweets...\n")
for i in range(nums[2],nums[3]):
    if( (i+1)%10000 == 0 ):
        print("Tweets %d of %d has been processed" % ( i+1, nums[3] ))
    clean_tweet_texts.append(tweet_cleaner(df['text'][i]))

Cleaning and parsing the tweets...

Tweets 810000 of 1200000 has been processed
Tweets 820000 of 1200000 has been processed
Tweets 830000 of 1200000 has been processed
Tweets 840000 of 1200000 has been processed
Tweets 850000 of 1200000 has been processed
Tweets 860000 of 1200000 has been processed
Tweets 870000 of 1200000 has been processed
Tweets 880000 of 1200000 has been processed
Tweets 890000 of 1200000 has been processed
Tweets 900000 of 1200000 has been processed
Tweets 910000 of 1200000 has been processed
Tweets 920000 of 1200000 has been processed
Tweets 930000 of 1200000 has been processed
Tweets 940000 of 1200000 has been processed
Tweets 950000 of 1200000 has been processed
Tweets 960000 of 1200000 has been processed
Tweets 970000 of 1200000 has been processed
Tweets 980000 of 1200000 has been processed
Tweets 990000 of 1200000 has been processed
Tweets 1000000 of 1200000 has been processed
Tweets 1010000 of 1200000 has been processed
Tweets 1020000 of 1200000 has been pro

In [27]:
%%time
print("Cleaning and parsing the tweets...\n")
for i in range(nums[3],nums[4]):
    if( (i+1)%10000 == 0 ):
        print("Tweets %d of %d has been processed" % ( i+1, nums[4] ))
    clean_tweet_texts.append(tweet_cleaner(df['text'][i]))

Cleaning and parsing the tweets...

Tweets 1210000 of 1600000 has been processed
Tweets 1220000 of 1600000 has been processed
Tweets 1230000 of 1600000 has been processed
Tweets 1240000 of 1600000 has been processed
Tweets 1250000 of 1600000 has been processed
Tweets 1260000 of 1600000 has been processed
Tweets 1270000 of 1600000 has been processed
Tweets 1280000 of 1600000 has been processed
Tweets 1290000 of 1600000 has been processed
Tweets 1300000 of 1600000 has been processed
Tweets 1310000 of 1600000 has been processed
Tweets 1320000 of 1600000 has been processed
Tweets 1330000 of 1600000 has been processed
Tweets 1340000 of 1600000 has been processed
Tweets 1350000 of 1600000 has been processed
Tweets 1360000 of 1600000 has been processed
Tweets 1370000 of 1600000 has been processed
Tweets 1380000 of 1600000 has been processed
Tweets 1390000 of 1600000 has been processed
Tweets 1400000 of 1600000 has been processed
Tweets 1410000 of 1600000 has been processed
Tweets 1420000 of 1

In [28]:
# save cleaned data
clean_df = pd.DataFrame(clean_tweet_texts, columns=['text'])
clean_df['target'] = df.sentiment
clean_clean_dfdf.head()

,text,target
0,"- awww, that's a bummer. you shoulda got da...",0
1,is upset that he can't update his facebook by ...,0
2,i dived many times for the ball. managed to s...,0
3,my whole body feels itchy and like its on fire,0
4,"no, it's not behaving at all. i'm mad. why am...",0


In [30]:
clean_df.tail()

,text,target
1599995,just woke up. having no school is the best fee...,1
1599996,thewdb.com - very cool to hear old walt interv...,1
1599997,are you ready for your mojo makeover? ask me f...,1
1599998,happy 38th birthday to my boo of alll time!!! ...,1
1599999,happy #charitytuesday,1


In [29]:
clean_df.to_csv('../data/twitter/clean_tweet_char.csv', encoding='utf-8')